## <Font color = 'indianred'>**1. Set Environment**

In [1]:
import sys

In [2]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output

  !pip install torchtext -qq
  !pip install transformers evaluate wandb datasets accelerate -U -qq
  !pip install -U nltk -qq
  !pip install -U spacy -qq
  !python -m spacy download en_core_web_sm -qq

  basepath = '/content/drive/MyDrive/data'
  sys.path.append('/content/drive/MyDrive/data/custom-functions')
else:
  basepath = 'D:/UTDallas/Courses/Spring 24/Applied NLP/Data Folder'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 

In [3]:
# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries
from torchtext.vocab import vocab

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np
import pandas as pd

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

### NEW ##########################
# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

In [4]:
base_folder = Path(basepath)
data_folder = base_folder / 'datasets/emotion_detection'
model_folder = base_folder / 'models/nlp_spring_2024/emotion_detection'

In [5]:
model_folder.mkdir(exist_ok=True, parents=True)
data_folder.mkdir(exist_ok=True, parents=True)

## <Font color = 'indianred'>**2. Load Dataset**

In [6]:
# location of train and test files
train_file = data_folder / 'train.csv'
test_file = data_folder / 'test.csv'

In [7]:
# creating Pandas Dataframe
train_data = pd.read_csv(train_file, index_col=0)
test_data = pd.read_csv(test_file, index_col=0)

In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7724 entries, 2017-21441 to 2018-03386
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tweet         7724 non-null   object
 1   anger         7724 non-null   int64 
 2   anticipation  7724 non-null   int64 
 3   disgust       7724 non-null   int64 
 4   fear          7724 non-null   int64 
 5   joy           7724 non-null   int64 
 6   love          7724 non-null   int64 
 7   optimism      7724 non-null   int64 
 8   pessimism     7724 non-null   int64 
 9   sadness       7724 non-null   int64 
 10  surprise      7724 non-null   int64 
 11  trust         7724 non-null   int64 
dtypes: int64(11), object(1)
memory usage: 784.5+ KB


In [9]:
y = train_data[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love','optimism', 'pessimism', 'sadness', 'surprise', 'trust']].to_numpy()

In [10]:
print(y)

[[0 1 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [1 0 1 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
X_train = train_data.drop(columns = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love','optimism', 'pessimism', 'sadness', 'surprise', 'trust'])
X_train.head()

,Tweet
ID,
2017-21441,“Worry is a down payment on a problem you may ...
2017-31535,Whatever you decide to do make sure it makes y...
2017-21068,@Max_Kellerman it also helps that the majorit...
2017-31436,Accept the challenges so that you can literall...
2017-22195,My roommate: it's okay that we can't spell bec...


In [12]:
from pprint import pprint
pprint(X_train.values[0], width = 100)

array(["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry"],
      dtype=object)


In [13]:
type(X_train.values[1])
print(X_train['Tweet'])

ID
2017-21441    “Worry is a down payment on a problem you may ...
2017-31535    Whatever you decide to do make sure it makes y...
2017-21068    @Max_Kellerman  it also helps that the majorit...
2017-31436    Accept the challenges so that you can literall...
2017-22195    My roommate: it's okay that we can't spell bec...
                                    ...                        
2018-01993    @BadHombreNPS @SecretaryPerry If this didn't m...
2018-01784    Excited to watch #stateoforigin tonight! Come ...
2018-04047    Blah blah blah Kyrie, IT, etc. @CJC9BOSS leavi...
2018-03041    #ThingsIveLearned The wise #shepherd never tru...
2018-03386    I am really flattered and happy to hear those ...
Name: Tweet, Length: 7724, dtype: object


In [14]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [15]:
import CustomPreprocessorSpacy as cp

In [16]:
X_train_len = len(X_train)
print(X_train_len)

7724


In [17]:
prefixes = list(nlp.Defaults.prefixes)
prefixes += ['@']
prefixes.remove(r'#')
prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [19]:
  # import spacy pre-processor from custom module
preprocessor = cp.SpacyPreprocessor(model='en_core_web_sm', batch_size=X_train_len, lemmatize=False, lower=False,
                                    remove_stop=False, remove_punct=False, remove_email=False,
                                    remove_url=False, remove_num=False, stemming=False,
                                    add_user_mention_prefix=False, remove_hashtag_prefix=False, basic_clean_only=True)

In [20]:
X_train_cleaned = preprocessor.fit_transform(list(X_train['Tweet']))

/content/drive/MyDrive/data/custom-functions/CustomPreprocessorSpacy.py:83: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [21]:
X_train_cleaned[:5]

["“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry",
 'Whatever you decide to do make sure it makes you #happy.',
 "@Max_Kellerman  it also helps that the majority of NFL coaching is inept. Some of Bill O'Brien's play calling was wow, ! #GOPATS",
 "Accept the challenges so that you can literally even feel the exhilaration of victory.' -- George S. Patton 🐶",
 "My roommate: it's okay that we can't spell because we have autocorrect. #terrible #firstworldprobs"]

In [22]:
type(X_train_cleaned)

list

In [23]:
X_train_cleaned[0]

"“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry"

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_cleaned, y, test_size = 0.3, random_state = 1)

In [25]:
trainset = Dataset.from_dict({
    'texts': X_train,
    'labels': y_train
})

validset = Dataset.from_dict({
    'texts': X_valid,
    'labels': y_valid
})

In [26]:
print(y_train.shape), print(y_valid.shape)

(5406, 11)
(2318, 11)


(None, None)

In [27]:
trainset[:5]

{'texts': ['@JuliaHB1 Bloody right #fume',
  'You boys dint know the game am I the game... life after death... better chose and know who side you on before my wrath does come upon us😤😤😤',
  'Peter is aesthetically pleasing to look at',
  'The weather changed from sunny and bright to gloomy just in time to match my afternoon mood. 😒',
  "#TerrorStatePak we r confirm that #navazsharif is post graduate distinction student of university of #terrorism. He can't spare himself."],
 'labels': [[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]]}

In [28]:
class CustomConfig(PretrainedConfig):
  def __init__(self, vocab_size=0, embedding_dim=0, hidden_dim1=0, hidden_dim2=0, num_labels=11, **kwargs):
      super().__init__()
      self.vocab_size = vocab_size
      self.embedding_dim = embedding_dim
      self.hidden_dim1 = hidden_dim1
      self.hidden_dim2 = hidden_dim2
      self.num_labels = num_labels

In [29]:
class CustomMLP(PreTrainedModel):
    config_class = CustomConfig

    def __init__(self, config):
        super().__init__(config)

        self.embedding_bag = nn.EmbeddingBag(config.vocab_size, config.embedding_dim)
        self.layers = nn.Sequential(
            nn.Linear(config.embedding_dim, config.hidden_dim1),
            nn.BatchNorm1d(num_features=config.hidden_dim1),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(config.hidden_dim1, config.hidden_dim2),
            nn.BatchNorm1d(num_features=config.hidden_dim2),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(config.hidden_dim2, config.num_labels)
        )

    def forward(self, input_ids, offsets, labels=None):
        embed_out = self.embedding_bag(input_ids, offsets)
        logits = self.layers(embed_out)

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits
        )

In [30]:
def get_vocab(dataset, min_freq=1):
    """
    Generate a vocabulary from a dataset.

    Args:
        dataset (Dataset): A Hugging Face Dataset object. The dataset should
                           have a key 'texts' that contains the text data.
        min_freq (int): The minimum frequency for a token to be included in
                        the vocabulary.

    Returns:
        torchtext.vocab.Vocab: Vocabulary object containing tokens from the
                               dataset that meet or exceed the specified
                               minimum frequency. It also includes a special
                               '<unk>' token for unknown words.
    """
    # Initialize a counter object to hold token frequencies
    counter = Counter()

    # Update the counter with tokens from each text in the dataset
    # Iterating through texts in the dataset
    for text in dataset['texts']:  ###### Change from previous function ####
        counter.update(str(text).split())

    # Create a vocabulary using the counter object
    # Tokens that appear fewer times than `min_freq` are excluded
    my_vocab = vocab(counter, min_freq=min_freq)

    # Insert a '<unk>' token at index 0 to represent unknown words
    my_vocab.insert_token('<unk>', 0)

    # Set the default index to 0
    # This ensures that any unknown word will be mapped to '<unk>'
    my_vocab.set_default_index(0)

    return my_vocab

In [31]:
# Creating a function that will be used to get the indices of words from vocab
def tokenizer(text, vocab):
    """Converts text to a list of indices using a vocabulary dictionary"""
    return [vocab[token] for token in str(text).split()]

In [32]:
def collate_batch(batch, my_vocab):
    """
    Prepares a batch of data by transforming texts into indices based on a vocabulary and
    converting labels into a tensor.

    Args:
        batch (list of dict): A batch of data where each element is a dictionary with keys
                              'labels' and 'texts'. 'labels' are the sentiment labels, and
                              'texts' are the corresponding texts.
        my_vocab (torchtext.vocab.Vocab): A vocabulary object that maps tokens to indices.

    Returns:
        dict: A dictionary with three keys:
              - 'input_ids': a tensor containing concatenated indices of the texts.
              - 'offsets': a tensor representing the starting index of each text in 'input_ids'.
              - 'labels': a tensor of the labels for each text in the batch.

    The function transforms each text into a list of indices based on the provided vocabulary.
    It also converts the labels into a tensor. The 'offsets' are computed to keep track of the
    start of each text within the 'input_ids' tensor, which is a flattened representation of all text indices.
    """

    # Get labels and texts from batch dict samples
    labels = [sample['labels'] for sample in batch]
    texts = [sample['texts'] for sample in batch]

    # Convert the list of labels into a tensor of dtype int32
    labels = torch.tensor(labels, dtype=torch.float64)

    # Convert the list of texts into a list of lists; each inner list contains the vocabulary indices for a text
    list_of_list_of_indices = [tokenizer(text, my_vocab) for text in texts]

    # Concatenate all text indices into a single tensor
    input_ids = torch.cat([torch.tensor(i, dtype=torch.int32) for i in list_of_list_of_indices])

    # Compute the offsets for each text in the concatenated tensor
    offsets = [0] + [len(i) for i in list_of_list_of_indices]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    # print('Labels:',labels.shape)
    # print('Inputs:',input_ids.shape)
    return {
        'input_ids': input_ids,
        'offsets': offsets,
        'labels': labels
    }

In [33]:
emotion_vocab = get_vocab(trainset, min_freq=2)
collate_fn = partial(collate_batch, my_vocab=emotion_vocab)
print(len(emotion_vocab))

7036


In [34]:
my_config = CustomConfig(vocab_size=len(emotion_vocab),
                         embedding_dim=300,
                         hidden_dim1=200,
                         hidden_dim2=100,
                         num_labels=11)


In [35]:
my_config.id2label = {0: 'anger', 1: 'anticipation', 2: 'disgust', 3: 'fear', 4: 'joy', 5: 'love', 6: 'optimism', 7: 'pessimism', 8: 'sadness' , 9: 'surprise', 10: 'trust'}

In [36]:
my_config.label2id = {v: k for k, v in my_config.id2label .items()}

In [37]:
my_config

CustomConfig {
  "embedding_dim": 300,
  "hidden_dim1": 200,
  "hidden_dim2": 100,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "transformers_version": "4.39.3",
  "vocab_size": 7036
}

In [38]:
model = CustomMLP(config=my_config)

In [39]:
model

CustomMLP(
  (embedding_bag): EmbeddingBag(7036, 300, mode='mean')
  (layers): Sequential(
    (0): Linear(in_features=300, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=200, out_features=100, bias=True)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=100, out_features=11, bias=True)
  )
)

In [40]:
def compute_metrics(eval_pred):
    combined_metrics = evaluate.combine([evaluate.load("accuracy"),
                                        evaluate.load("f1", average="macro")])

    logits, labels = eval_pred
    predictions = (logits >= 0).astype(float)
    predictions = predictions.reshape(-1)
    labels = labels.reshape(-1)
    evaluations = combined_metrics.compute(
        predictions=predictions, references=labels,)
    return evaluations

In [41]:
# Configure training parameters
training_args = TrainingArguments(

    # Training-specific configurations
    num_train_epochs=5,
    per_device_train_batch_size=128, # Number of samples per training batch
    per_device_eval_batch_size=128, # Number of samples per validation batch
    weight_decay=0.1, # weight decay (L2 regularization)
    learning_rate=0.001, # learning arte
    optim='adamw_torch', # optimizer
    remove_unused_columns=False, # flag to retain unused columns

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=50,  # Perform evaluation every 50 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=50,  # Save a model checkpoint every 50 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=2,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="accuracy",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations
    logging_strategy='steps',
    logging_steps=50,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='emotion_hf_trainer',  # Experiment name for Weights & Biases
)

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset = validset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset = validset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [45]:
!wandb login

wandb: Currently logged in as: dakavyshnavi (vyshnavi-utd). Use `wandb login --relogin` to force relogin


In [46]:
# specify the project name where the experiment will be logged
%env WANDB_PROJECT = nlp_course_spring_2024-emotion-detection-hf-trainer

env: WANDB_PROJECT=nlp_course_spring_2024-emotion-detection-hf-trainer


In [47]:
trainer.train()

wandb: Currently logged in as: dakavyshnavi (vyshnavi-utd). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,F1
50,0.587200,0.542030,0.785591,0.036312
100,0.493500,0.491591,0.789081,0.051499
150,0.473300,0.479186,0.789670,0.090863
200,0.463700,0.476273,0.790062,0.127181


TrainOutput(global_step=215, training_loss=0.5016522496245628, metrics={'train_runtime': 23.7752, 'train_samples_per_second': 1136.901, 'train_steps_per_second': 9.043, 'total_flos': 5691331309950.0, 'train_loss': 0.5016522496245628, 'epoch': 5.0})

In [48]:
trainer.evaluate()

{'eval_loss': 0.4762727618217468,
 'eval_accuracy': 0.7900619656443643,
 'eval_f1': 0.12718082504483938,
 'eval_runtime': 3.7227,
 'eval_samples_per_second': 622.665,
 'eval_steps_per_second': 5.104,
 'epoch': 5.0}

In [49]:
valid_output = trainer.predict(validset)

In [50]:
valid_output._fields

('predictions', 'label_ids', 'metrics')

In [51]:
valid_output.predictions

array([[-0.6834613 , -1.1398375 , -0.5072583 , ..., -0.93758214,
        -1.7102088 , -2.2173314 ],
       [-0.20302682, -1.6795335 , -0.26163396, ..., -0.89466506,
        -2.277391  , -2.2901163 ],
       [-0.29971167, -1.0215217 , -0.18628983, ..., -0.8811201 ,
        -1.6641756 , -1.7471274 ],
       ...,
       [-0.4935178 , -0.9766959 , -0.86531854, ..., -0.684955  ,
        -1.7964075 , -2.08717   ],
       [-0.3097919 , -1.8157611 , -0.11740586, ..., -0.40608302,
        -1.8524956 , -1.9234427 ],
       [-0.2719638 , -1.1834182 , -0.44707105, ..., -0.75554043,
        -1.8009778 , -1.8279911 ]], dtype=float32)

In [52]:
valid_preds = (valid_output.predictions >= 0).astype(float)
valid_labels = np.array(valid_output.label_ids)